# Using Whisper in Google Colab
This notebook provides a simple template for using OpenAI's Whisper for audio transcription in Google Colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1UKDLaTbhwJ3LnrKmIYbHcW7R3KomTBma)
## Install Whisper
Run the cell below to install Whisper.

In [ ]:
!pip install git+https://github.com/openai/whisper.git

##Option 1: Upload Your Audio File
Use the file upload feature of Google Colab to upload your audio file.

Also, a recording of the author's voice can be found at Packt's GitHub repository:

https://github.com/PacktPublishing/Learn-OpenAI-Whisper/blob/main/Chapter01/Learn_OAI_Whisper_Sample_Audio01.m4a

### Transcribe audio
If uploaded file, replace 'your_audio_file_here' with your file's name and run the cell to transcribe.

In [ ]:
!whisper your_audio_file_here --model small.en

# Oprion 2: Download sample files

In [ ]:
# Install gdown library, allows downloading of files from Google Drive - https://pypi.org/project/gdown/
!pip install --upgrade gdown

In [ ]:
# Download using Google Drive Ids

file_id1 = '11ur787d87Z49luASqxNUqyiRsEIAR0O7'
file_id2 = '1gAcVBDBO9MJw-8TEHGORRd0cKjrRtQEY'

!gdown $file_id1 -O Learn_OAI_Whisper_Sample_Audio01.mp3
!gdown $file_id2 -O Learn_OAI_Whisper_Sample_Audio02.mp3

In [ ]:
mono_file = "Learn_OAI_Whisper_Sample_Audio01.mp3"
stereo_file = "Learn_OAI_Whisper_Sample_Audio02.mp3"

In [ ]:
import whisper

# Load the small English model
model = whisper.load_model("small")

In [ ]:
# Transcribe the mono audio file
result = model.transcribe(mono_file)
print("Transcription of mono_file:")
print(result["text"])

In [ ]:
# Transcribe the stereo audio file
result = model.transcribe(stereo_file)
print("Transcription of stereo_file:")
print(result["text"])